In [ ]:
import os

http://cas.sdss.org/dr7/en/tools/crossid/crossid.asp
上述网站可以根据ra，dec返回run，rerun，camcol，field信息，这些信息可以根据对应在http://das.sdss.org/raw/中对应下载到
但是上述网站一次只能上传100KB的文件，经测试，200KB是最大承受范围，下面用来切割坐标列表，7000一组

In [ ]:
fits_dir = "/data/renhaoye/decals_2022/in_decals/fits/"
fits_files = os.listdir(fits_dir)
fits_files.sort()
step = 7000
iter = int(len(fits_files) / 7000)
for batch in range(iter + 1):
    with open("/data/renhaoye/decals_2022/decals_poi_7k%d.txt" % batch, "w") as f:
        if batch == iter:
            for i in range(batch * step, len(fits_files)):
                poi = fits_files[i].split(".fits")[0]
                ra, dec = poi.split("_")
                f.write("A" + str(i + 1) + " " + str(ra) + " " + str(dec) + "\n")
        else:
            for i in range(batch * step, (batch + 1) * step):
                poi = fits_files[i].split(".fits")[0]
                ra, dec = poi.split("_")
                # f.write(str(ra) + "," + str(dec) + "," + "1.117866\n")
                f.write("A" + str(i + 1) + " " + str(ra) + " " + str(dec) + "\n")

切割图像

In [ ]:
import numpy as np
import astropy.units as u
import astropy
import pandas as pd
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy.io import fits
from astroquery.gaia import Gaia
import matplotlib.pyplot as plt
Gaia.MAIN_GAIA_TABLE = "gaiaedr3.gaia_source" # Select early Data Release 3
%matplotlib inline

In [ ]:
PATH = "/data/renhaoye/decals_2022/"
hdul = fits.open(PATH + "g.fit")

data = hdul[0].data
header = hdul[0].header
wcsobj = WCS(hdul[0].header)
hdul.close()

In [ ]:
ra, dec = wcsobj.all_world2pix([[122.541511, 20.486553]], 0)[0]

In [ ]:
sdss = pd.read_csv("/data/renhaoye/decals_2022/sdss_in_decals.csv")

In [ ]:
x = sdss.iloc[1]
type(x)

In [ ]:
sdss.info()

In [172]:
import warnings
warnings.filterwarnings("ignore")
class Cut:
    def __init__(self, info):
        assert type(info)==pd.core.series.Series, "please input pd.core.series.Series like: df.iloc[index]"
        self.ra = info.ra
        self.dec = info.dec
        self.run = info.run
        self.rerun = info.rerun
        self.camcol = info.camcol
        self.field = info.field
        self.cutout = np.zeros((3, 256, 256))
    def __call__(self, *args, **kwargs):
        if len(str(self.field)) == 2:
            self.filed = "0" + self.field
        raw_path = "/data/renhaoye/sdss_dr7_decals_overlap/raw/"
        bands = ["g", "r", "z"]
        # files = []
        # for band in bands:
        #     files.append(raw_path + "fpC-00%s-%s%d-0%s.fit" % (self.run, band, self.camcol, self.field))
        files = ["/data/renhaoye/decals_2022/g.fit", "/data/renhaoye/decals_2022/r.fit", "/data/renhaoye/decals_2022/z.fit"]
        for i in range(len(files)):
            with fits.open(files[i]) as hdul:
                data = hdul[0].data
                wcs = WCS(hdul[0].header)
                # x, y = wcs.all_world2pix([[self.ra, self.dec]],0)[0]
                y, x = wcs.all_world2pix([[122.541511, 20.486553]], 0)[0]
                # assert x == np.nan or y == np.nan, "source not found in fit"
                x, y = int(x), int(y)
                left_upper_x, left_upper_y = x-128, y-128
                self.cutout[i] = data[left_upper_x:left_upper_x + 256, left_upper_y:left_upper_y+256]

In [173]:
a = Cut(x)
a()
print(a.cutout)

[[[1060. 1067. 1068. ... 1060. 1065. 1075.]
  [1072. 1068. 1072. ... 1065. 1074. 1067.]
  [1061. 1066. 1064. ... 1076. 1068. 1071.]
  ...
  [1064. 1075. 1066. ... 1074. 1063. 1067.]
  [1066. 1068. 1063. ... 1070. 1066. 1064.]
  [1061. 1076. 1070. ... 1070. 1068. 1066.]]

 [[1113. 1105. 1108. ... 1100. 1103. 1107.]
  [1113. 1112. 1108. ... 1108. 1101. 1104.]
  [1108. 1108. 1109. ... 1106. 1107. 1109.]
  ...
  [1111. 1112. 1099. ... 1100. 1095. 1114.]
  [1100. 1112. 1113. ... 1115. 1111. 1100.]
  [1103. 1103. 1108. ... 1112. 1108. 1103.]]

 [[1105. 1102. 1098. ... 1093. 1089. 1090.]
  [1094. 1104. 1101. ... 1103. 1104. 1104.]
  [1112. 1101. 1090. ... 1101. 1104. 1101.]
  ...
  [1103. 1104. 1099. ... 1094. 1101. 1094.]
  [1100. 1106. 1105. ... 1105. 1098. 1094.]
  [1100. 1101. 1101. ... 1105. 1095. 1092.]]]


In [174]:
def save_fits(data: np.ndarray, filename: str):
    """
    将ndarray保存成fits文件
    :param data: 待保存数据
    :param filename: 保存文件名
    :return:
    """
    if len(data.shape) == 2:
        hdu = fits.PrimaryHDU(data)
        hdulist = fits.HDUList([hdu])
        hdulist.writeto(filename)
        hdulist.close()
    elif data.shape[-1] == 3:
        g, r, z = data[:, :, 0], data[:, :, 1], data[:, :, 2]
        data = np.array((g, r, z))
        hdu = fits.PrimaryHDU(data)
        hdulist = fits.HDUList([hdu])
        hdulist.writeto(filename)
        hdulist.close()
    elif data.shape[0] == 3:
        hdu = fits.PrimaryHDU(data)
        hdulist = fits.HDUList([hdu])
        hdulist.writeto(filename)
        hdulist.close()
    else:
        raise RuntimeError

In [175]:
save_fits(a.cutout, "/data/renhaoye/1.fits")